# Homework 4 (Sequential Models)

1. Choose a book or other text to train your model on (I suggest [Project Gutenberg](https://www.gutenberg.org/ebooks/) to find .txt files but you can find them elsewhere). Make sure your file is a `.txt` file. Clean your data (You may use [this file](https://colab.research.google.com/drive/1HCgKn5XQ7Q3ywxGszVWx2kddfT9UBASp?usp=sharing) we talked about in class as a baseline). Build a sequential model (LSTM, GRU, SimpleRNN, or Transformer) that generates new lines based on the training data (NO TRANSFER LEARNING).

Print out or write 10 generated sequences from your model (Similar to Classwork 17 where we generated new Pride and Prejudice lines, but now with words instead of charachters. Feel free to use [this](https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/) as a reference). Assess in detail how good they are, what they're good at, what they struggle to do well. 

2. Make a new model with ONE substantial adjustment (e.g. use a custom embedding layer if you didn't already, use a pre-trained embedding layer if you didn't already, use a DEEP LSTM/GRU with multiple recurrent layers, use a pre-trained model to do transfer learning and fine-tune it...etc.). 

Print out or write 10 generated sequences from your model (Similar to Classwork 17 where we generated new Pride and Prejudice lines, but now with words instead of charachters. Feel free to use [this](https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/) as a reference). Assess in detail how good they are, what they're good at, what they struggle to do well.  Did the performance of your model change?

3. Then create a **technical report** discussing your model building process, the results, and your reflection on it. The report should follow the format in the example including an Introduction, Analysis, Methods, Results, and Reflection section.

In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string

my_file = '/Users/isaacmin/Desktop/code/philosopherstone.txt'
seq_len = 100

def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

my_file_seq ='/Users/isaacmin/Desktop/code/philosopherstone.txt'
doc = load_doc(my_file)
print(doc[:200])

# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = seq_len + 1
sequences = list()
for i in range(length, len(tokens)):
 # select sequence of tokens
 seq = tokens[i-length:i]
 # convert into a line
 line = ' '.join(seq)
 # store
 sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = my_file[:-4] + '_sequences.txt'
save_doc(sequences, out_filename)



THE BOY WHO LIVED 

Mr. and Mrs. Dursley, of number four, Privet Drive, 
were proud to say that they were perfectly normal, 
thank you very much. They were the last people you’d 
expect to be involved
['the', 'boy', 'who', 'lived', 'mr', 'and', 'mrs', 'dursley', 'of', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', 'thank', 'you', 'very', 'much', 'they', 'were', 'the', 'last', 'people', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'hold', 'with', 'such', 'nonsense', 'mr', 'dursley', 'was', 'the', 'director', 'of', 'a', 'firm', 'called', 'grunnings', 'which', 'made', 'drills', 'he', 'was', 'a', 'big', 'beefy', 'man', 'with', 'hardly', 'any', 'neck', 'although', 'he', 'did', 'have', 'a', 'very', 'large', 'mustache', 'mrs', 'dursley', 'was', 'thin', 'and', 'blonde', 'and', 'had', 'nearly', 'twice', 'the', 'usual', 'amount', 'of', 'neck', 'which', 'came', 'in',

In [27]:
# load
in_filename = '/Users/isaacmin/Desktop/code/philosopherstone_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

# create a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes = vocab_size)
seq_length = X.shape[1]

p_train = 0.8

n_train = int(X.shape[0]//(1/p_train))
X_train = X[0:n_train]
y_train = y[0:n_train]
X_test = X[n_train:]
y_test = y[n_train:]


In [28]:
X_train.shape

(58833, 100)

In [29]:
X_test.shape

(14709, 100)

In [30]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [31]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [32]:
y_train.shape

(58833, 5562)

In [33]:
y_test.shape

(14709, 5562)

In [34]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = seq_length))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(vocab_size, activation = 'softmax'))
print(model.summary())

2023-04-26 15:31:41.311849: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           278100    
                                                                 
 lstm (LSTM)                 (None, 100, 100)          60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 5562)              561762    
                                                                 
Total params: 990,762
Trainable params: 990,762
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
# comile model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# fit model
model.fit(X, y, batch_size = 128, epochs = 100)

Epoch 1/100
575/575 [==============================] - 147s 251ms/step - loss: 5.6792 - accuracy: 0.1241
Epoch 2/100
575/575 [==============================] - 140s 244ms/step - loss: 5.4868 - accuracy: 0.1307
Epoch 3/100
575/575 [==============================] - 143s 249ms/step - loss: 5.3590 - accuracy: 0.1359
Epoch 4/100
575/575 [==============================] - 145s 252ms/step - loss: 5.2604 - accuracy: 0.1398
Epoch 5/100
575/575 [==============================] - 143s 249ms/step - loss: 5.1636 - accuracy: 0.1446
Epoch 6/100
575/575 [==============================] - 139s 242ms/step - loss: 5.0780 - accuracy: 0.1472
Epoch 7/100
575/575 [==============================] - 140s 243ms/step - loss: 5.0024 - accuracy: 0.1512
Epoch 8/100
575/575 [==============================] - 140s 244ms/step - loss: 4.9270 - accuracy: 0.1546
Epoch 9/100
575/575 [==============================] - 169s 294ms/step - loss: 4.8612 - accuracy: 0.1571
Epoch 10/100
575/575 [==============================] -

KeyboardInterrupt: 